## extracting from recon3d

In [1]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv


MASSpy version: 0.1.6


In [2]:
#load Recon3D
model_dir = os.path.abspath("models")
data_dir = os.path.abspath("data")
# model_dir
R3D=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))
core= load_json_cobra_model(filename=os.path.join(model_dir,"CoreModel.json"))



Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-04


In [3]:
_REQUIRED_REACTION_ATTRIBUTES = [
    "id",
    "name",
    "metabolites",
    "lower_bound",
    "upper_bound",
#     "gene_reaction_rule",
]

_REQUIRED_METABOLITE_ATTRIBUTES = ["id", "name", "charge","formula", "compartment"]
_ORDERED_OPTIONAL_METABOLITE_KEYS = [
   
    "_bound",
    "notes",
    "annotation",
]


In [4]:
# def metabolite_to_dict(metabolite):
#     new_met = dict()
#     for key in _REQUIRED_METABOLITE_ATTRIBUTES:
#         new_met[key] = _fix_type(getattr(metabolite, key))
#     return new_met
from cobra_dict import metabolite_to_dict as metabolite_to_dict
from cobra_dict import reaction_to_dict as reaction_to_dict

In [5]:
reaction_list=[]
for rid in core.reactions:
    reaction_list.append(rid.id)
len(reaction_list)

187

In [6]:
metabolite_list=[]
for mid in core.metabolites:
    metabolite_list.append(mid.id)
len(metabolite_list)

178

In [7]:
#Function to add underscore in front of metabolite identifiers which start with a number
def prefix_number_id(id_str):
    """Prefix identifiers that start with numbers."""
    if re.match(r"^\d", id_str):
        id_str = "_" + id_str
    return id_str

In [ ]:
# #Loop to edit the names using "prefix_number_id" function defined earlier
# for metabolite in glycolysis_ppp_tca_etc.metabolites:
#     new_met_id = prefix_number_id(metabolite.id)
#     metabolite.id = new_met_id
# glycolysis_ppp_tca_etc.repair()

In [26]:
met_df=pd.DataFrame()
for met in metabolite_list:
    if met in R3D.metabolites:
        r3d_met= R3D.metabolites.get_by_id(met)
        m=metabolite_to_dict(r3d_met)

    else:
        core_met = core.metabolites.get_by_id(met)
        m=metabolite_to_dict(core_met)
    df_2=pd.DataFrame.from_dict(m,orient='index')
    df_2=df_2.T
    met_df= pd.concat([met_df, df_2])

met_df=met_df.set_index('id')
met_df.loc["h2o_c"]

name           H2O H2O
charge               0
formula            H2O
compartment          c
Name: h2o_c, dtype: object

In [11]:
csv_met = os.path.join(data_dir,"big_core_met_df.csv")
met_df.to_csv(csv_met)

In [75]:
metabolite_list=[]
met_df=pd.DataFrame()
rxn_df=pd.DataFrame()
reactions_not_in_r3d=[]
for rxn in reaction_list:
    if rxn in R3D.reactions:
        reaction= R3D.reactions.get_by_id(rxn)
        r=reaction_to_dict(reaction)
    else:
        reaction = core.reactions.get_by_id(rxn)
        r=reaction_to_dict(reaction)
        reactions_not_in_r3d.append(rxn)
        
    df=pd.DataFrame.from_dict(r,orient='index')
    df=df.T
    rxn_df= pd.concat([rxn_df, df])
    for met in reaction.metabolites:
        if met.id not in metabolite_list:
            if met.id in R3D.metabolites:
                metabolite= R3D.metabolites.get_by_id(met.id)
                m=metabolite_to_dict(metabolite)

            else:
                metabolite = core.metabolites.get_by_id(met.id)
                m=metabolite_to_dict(metabolite)
            # print(met.id)
            metabolite_list.append(met.id)
            df_2=pd.DataFrame.from_dict(m,orient='index')
            df_2=df_2.T
            met_df= pd.concat([met_df, df_2])


rxn_df=rxn_df.set_index('id')
met_df=met_df.set_index('id')
rxn_df
# reactions_not_in_r3d

,name,metabolites,lower_bound,upper_bound,gene_reaction_rule,subsystem,notes,annotation
id,,,,,,,,
CSm,Citrate synthase,"{'accoa_m': -1.0, 'cit_m': 1.0, 'coa_m': 1.0, ...",0.0,1000.0,1431_AT1 or 1431_AT2,Citric acid cycle,{'original_bigg_ids': ['CSm']},"{'bigg.reaction': ['CSm'], 'biocyc': ['META:CI..."
ACONTm,Aconitate hydratase,"{'cit_m': -1.0, 'icit_m': 1.0}",-1000.0,1000.0,50_AT1 or 48_AT1,Citric acid cycle,{'original_bigg_ids': ['ACONTm']},"{'bigg.reaction': ['ACONTm'], 'ec-code': ['4.2..."
ICDHxm,Isocitrate dehydrogenase NAD,"{'akg_m': 1.0, 'co2_m': 1.0, 'icit_m': -1.0, '...",0.0,1000.0,(3421_AT2 and 3420_AT3 and 3419_AT1) or (3420_...,Citric acid cycle,{'original_bigg_ids': ['ICDHxm']},"{'bigg.reaction': ['ICDHxm'], 'ec-code': ['1.1..."
AKGDm,2-oxoglutarate dehydrogenase,"{'akg_m': -1.0, 'co2_m': 1.0, 'coa_m': -1.0, '...",0.0,1000.0,(4967_AT2 and 1738_AT1 and 8050_AT1 and 1743_A...,Citric acid cycle,{'original_bigg_ids': ['AKGDm']},"{'bigg.reaction': ['AKGDm'], 'biocyc': ['META:..."
SUCOASm,Succinate CoA ligase ADP forming,"{'adp_m': 1.0, 'atp_m': -1.0, 'coa_m': -1.0, '...",-1000.0,1000.0,8802_AT1 and 8803_AT1,Citric acid cycle,{'original_bigg_ids': ['SUCOASm']},"{'bigg.reaction': ['SUCOASm'], 'biocyc': ['MET..."
...,...,...,...,...,...,...,...,...
EX_nh4_c,,{'nh4_c': -1},-1000.0,0.0,,NaN,NaN,NaN
EX_so3_c,,{'so3_c': -1},0.0,1000.0,,NaN,NaN,NaN
EX_etoh_c,,{'etoh_c': -1},0.0,1000.0,,NaN,NaN,NaN


In [63]:
len(metabolite_list)

182

In [74]:
"h2o2_c" in met_df.index

False

In [32]:
reactions_not_in_r3d

['Htm',
 'H2Otr',
 'PItm',
 'PItr',
 'PEPtm',
 'FACOAL80PROD',
 'GAPT',
 'AGPTAT',
 'LPIN',
 'MCOATA',
 'ACOATA',
 'KAS14',
 '30AR40',
 '3HAD40',
 'EAR40y',
 '30AS60',
 '30AR60',
 '3HAD60',
 'EAR60y',
 '30AS80',
 '30AR80',
 '3HAD80',
 'EAR80y',
 'FA80ACPHi',
 'ACOAD3fm',
 'ECOAH3m',
 'HACD3m',
 'ACACT3rm',
 'ACOAD2fm',
 'ECOAH2m',
 'HACD2m',
 'ACACT2rm',
 'ACACT1rm',
 'EX_h_c',
 'EX_pyr_c',
 'EX_h2o_c',
 'EX_pi_c',
 'EX_glc__D_c',
 'EX_lac__L_c',
 'EX_co2_c',
 'EX_o2_c',
 'EX_octa_prod_c',
 'EX_octa_cons_c',
 'EX_urea_c',
 'EX_gln__L_c',
 'EX_acetone_c',
 'EX_bhb_c',
 'EX_glu__L_c',
 'EX_ser__L_c',
 'EX_cys__L_c',
 'EX_gly_c',
 'EX_glygn2_c',
 'EX_Tyr_ggn_c',
 'EX_ala__L_c',
 'EX_nh4_c',
 'EX_so3_c',
 'EX_etoh_c',
 'EX_glyc_3octa_c',
 'EX_fru_c']

In [18]:
# to_b_added = ['Htm',
#  'H2Otr',
#  'PItm',
#  'PItr',
#  'PEPtm',
#  'FACOAL80PROD',
#  'GAPT',
#  'AGPTAT',
#  'LPIN',
#  'MCOATA',
#  'ACOATA',
#  'KAS14',
#  '30AR40',
#  '3HAD40',
#  'EAR40y',
#  '30AS60',
#  '30AR60',
#  '3HAD60',
#  'EAR60y',
#  '30AS80',
#  '30AR80',
#  '3HAD80',
#  'EAR80y',
#  'FA80ACPHi',
#  'ACOAD3fm',
#  'ECOAH3m',
#  'HACD3m',
#  'ACACT3rm',
#  'ACOAD2fm',
#  'ECOAH2m',
#  'HACD2m',
#  'ACACT2rm',
#  'ACACT1rm']

In [65]:
# for i in to_b_added:
csv_met = os.path.join(data_dir,"big_core_met_df.csv")
met_df.to_csv(csv_met)
    

In [71]:
met_df.loc["h2o2_c"]

KeyError: 'h2o2_c'

In [67]:
csv_rxn = os.path.join(data_dir,"big_core_reaction_df.csv")
rxn_df.to_csv(csv_rxn)

In [68]:
rxn_df.index

Index(['CSm', 'ACONTm', 'ICDHxm', 'AKGDm', 'SUCOASm', 'SUCD1m', 'FUMm', 'MDHm',
       'NADH2_u10mi', 'FADH2ETC',
       ...
       'EX_cys__L_c', 'EX_gly_c', 'EX_glygn2_c', 'EX_Tyr_ggn_c', 'EX_ala__L_c',
       'EX_nh4_c', 'EX_so3_c', 'EX_etoh_c', 'EX_glyc_3octa_c', 'EX_fru_c'],
      dtype='object', name='id', length=187)

In [69]:
rxn_df.index.is_unique

True

In [24]:
json_rxn = os.path.join(data_dir,"big_core_reaction_df.json")
rxn_df.to_json(json_rxn
,orient="index"
)   